In [2]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/kaggle/input/clickbait-new/dataset/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet, 'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [3]:
test_dataset

,text,labels
0,Study shows that most Thais spend majority of ...,False
1,"Xbox One's Terraria Facing This ""Serious Issue""",True
2,The Simple Way To Banish Stinky Garlic Breath,True
3,He Went To Rip Weeds From His Lawn. Hours Late...,False
4,The Dish You Should Never Order in Rome,True
...,...,...
250,"Dog Dies One Hour After Hiking With His Owner,...",False
251,This is what happens when you leave a hotel cl...,False
252,This Texas GOP elector announces that he won't...,True
253,WikiLeaks' Julian Assange Reported Dead,False


In [4]:
train_dataset

,text,labels
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [5]:
validation_dataset

,text,labels
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
397,Target impresses with new designer collaboration,True
398,Research Reveals Indian Men Are Becoming Infer...,False
399,Jennifer Lawrence Knows Exactly What She Would...,True
400,Hillary Clinton Is Advertising in Deep-Red Neb...,False


# Training

In [3]:
# configurations = []

# for learn_rate in [1e-5, 4e-5, 1e-4]:
#     for warumup_ratio in [0.02, 0.06, 0.1]:
#         for batch_size in [8, 16, 32]:
#             configurations += [{
#                 "overwrite_output_dir": True,
#                 "num_train_epochs": 10,
#                 "fp16": False,
#                 "train_batch_size": batch_size,
#                 "gradient_accumulation_steps": 4,
#                 "evaluate_during_training": True,
#                 "max_seq_length": 64,
#                 "learning_rate": learn_rate,
#                 "early_stopping_consider_epochs": True,
#                 "early_stopping_delta": 0.01,
#                 "early_stopping_metric": "acc",
#                 "early_stopping_metric_minimize": False,
#                 "early_stopping_patience": 3,
#                 "evaluate_during_training_steps": 331,
#                 "output_dir": "outputs/",
#                 'warmup_ratio': warumup_ratio,
#                 # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
#                 #                    # <> überschreiben sich gegenseitig
#                 'save_steps': 2000,
#                 "manual_seed": 12345
#             }]

In [7]:
# def train_model(config, num):
#     from simpletransformers.classification import ClassificationModel
#     import sklearn
    
#     config["output_dir"] = "outputs/deberta_" + str(num)
    
#     model = ClassificationModel("deberta", "microsoft/deberta-large", args = config)
#     # model.train_model(train_dataset)
#     model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [ ]:
# for config, num in zip(configurations, range(len(configurations))):
#     print('Running configuration number', num)
    
#     train_model(config, num)

In [8]:
# !pip install simpletransformers

In [9]:
from simpletransformers.classification import ClassificationModel
import sklearn

args = {"output_dir": "outputs/test",
        "overwrite_output_dir": True,
        "num_train_epochs": 2,
        "fp16": False,
        "train_batch_size": 8,
        "gradient_accumulation_steps": 4,
        "evaluate_during_training": True,
        "max_seq_length": 64,
        "learning_rate": 4e-5,
        "early_stopping_consider_epochs": True,
        "early_stopping_delta": 0.01,
        "early_stopping_metric": "acc",
        "early_stopping_metric_minimize": False,
        "early_stopping_patience": 3,
        "evaluate_during_training_steps": 331,
        "output_dir": "outputs/",
        'warmup_ratio': 0.06,
        # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
        #                    # <> überschreiben sich gegenseitig
        'save_steps': 0,
        "manual_seed": 12345
        }

model = ClassificationModel("deberta", "microsoft/deberta-large", args=args)
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Downloading:   0%|          | 0.00/475 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['classifier.weight', 'classi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Running Epoch 0 of 2:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/402 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/402 [00:00<?, ?it/s]

(164,
 defaultdict(list,
             {'global_step': [82, 164],
              'train_loss': [0.5298047661781311, 0.09994448721408844],
              'mcc': [0.5745003577850173, 0.6125883470123705],
              'tp': [141, 151],
              'tn': [172, 171],
              'fp': [22, 23],
              'fn': [67, 57],
              'auroc': [0.8479381443298969, 0.8707870737509914],
              'auprc': [0.8579301031684478, 0.889676032129078],
              'acc': [0.7786069651741293, 0.8009950248756219],
              'eval_loss': [0.48884588363123876, 0.44092345705219343]}))

In [10]:
model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/402 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/51 [00:00<?, ?it/s]

({'mcc': 0.6125883470123705,
  'tp': 151,
  'tn': 171,
  'fp': 23,
  'fn': 57,
  'auroc': 0.8707870737509914,
  'auprc': 0.889676032129078,
  'acc': 0.8009950248756219,
  'eval_loss': 0.44092345705219343},
 array([[-3.69448811e-01,  4.50969726e-01],
        [-2.00113034e+00,  1.76592839e+00],
        [ 9.13081825e-01, -6.87205613e-01],
        [-1.83740866e+00,  1.56203663e+00],
        [-1.12956631e+00,  1.09379816e+00],
        [ 7.43996859e-01, -6.41823053e-01],
        [ 1.74168658e+00, -9.57033038e-01],
        [ 7.76495039e-01, -7.14572489e-01],
        [-2.08652592e+00,  1.65763009e+00],
        [ 1.29454851e+00, -6.21822476e-01],
        [ 8.28791499e-01, -5.66053569e-01],
        [ 1.83199978e+00, -1.10853875e+00],
        [ 5.02643466e-01, -2.65105873e-01],
        [ 5.55500507e-01, -4.54481512e-01],
        [ 4.14034665e-01, -3.20261955e-01],
        [ 4.11208332e-01, -2.26841807e-01],
        [ 1.95499289e+00, -9.79984999e-01],
        [ 1.32221889e+00, -7.14078903e-01],
  

In [11]:
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/255 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

({'mcc': 0.49434014284916006,
  'tp': 86,
  'tn': 104,
  'fp': 24,
  'fn': 41,
  'auroc': 0.8225270669291339,
  'auprc': 0.8512019432170588,
  'acc': 0.7450980392156863,
  'eval_loss': 0.5193588202819228},
 array([[ 0.37633628, -0.23835011],
        [ 0.54573762, -0.13413818],
        [ 0.14886276, -0.05759223],
        [ 0.19049934, -0.19920743],
        [-0.97461474,  1.07655251],
        [ 0.94092214, -0.57689244],
        [ 1.21011674, -0.59921932],
        [ 0.31809211, -0.12690766],
        [ 0.8255313 , -0.77975571],
        [ 0.76565081, -0.67043519],
        [ 0.07499927,  0.04217346],
        [-2.46137094,  2.06943417],
        [ 0.50956738, -0.21944804],
        [-2.39788032,  1.94155395],
        [ 0.02745128,  0.02578604],
        [ 1.09877503, -0.45916316],
        [ 1.09758031, -0.76028216],
        [-2.42046452,  2.08028412],
        [-2.23352885,  1.78449059],
        [ 1.13491154, -0.52442241],
        [ 0.32838359, -0.13705629],
        [ 0.65998197, -0.30848587],
  

# Model Selection with Validation data

In [ ]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm
import os.path


for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/deberta_" + str(num) + "/"
    
    df = []
    
    if os.path.isfile(directory + 'eval-results.jsonl'):
        print('Skip ', num)
        continue
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        # print(checkpoint)
        model = ClassificationModel("deberta", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
    
    pd.DataFrame(df).to_json(directory + 'eval-results.jsonl')



In [5]:
import pandas as pd

df = []
for config, num in zip(configurations, range(len(configurations))):
    df += [pd.read_json("outputs/deberta_" + str(num) + "/eval-results.jsonl")]
    
df = pd.concat(df)
df

,checkpoint,valid_acc,test_acc,config
0,outputs/deberta_0/checkpoint-246-epoch-3,0.779300,0.778450,"{'overwrite_output_dir': True, 'num_train_epoc..."
1,outputs/deberta_0/checkpoint-656-epoch-8,0.768645,0.750605,"{'overwrite_output_dir': True, 'num_train_epoc..."
2,outputs/deberta_0/checkpoint-820-epoch-10,0.786910,0.755448,"{'overwrite_output_dir': True, 'num_train_epoc..."
3,outputs/deberta_0/checkpoint-410-epoch-5,0.779300,0.766344,"{'overwrite_output_dir': True, 'num_train_epoc..."
4,outputs/deberta_0/checkpoint-331,0.756469,0.755448,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
5,outputs/deberta_26/checkpoint-80-epoch-4,0.770167,0.773608,"{'overwrite_output_dir': True, 'num_train_epoc..."
6,outputs/deberta_26/checkpoint-200-epoch-10,0.788432,0.771186,"{'overwrite_output_dir': True, 'num_train_epoc..."
7,outputs/deberta_26/checkpoint-180-epoch-9,0.786910,0.755448,"{'overwrite_output_dir': True, 'num_train_epoc..."
8,outputs/deberta_26/checkpoint-20-epoch-1,0.657534,0.690073,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [ ]:
df.to_json('outputs/deberta_eval_results.csv')

In [5]:
len(configurations)

27

In [ ]:
print('finished!')

In [ ]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

In [6]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.8066971081000001 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmup_ratio': 0.02, 'save_steps': 2000, 'manual_seed': 12345}

0.797564688 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 16, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmup_ratio